In [50]:
#Install packages
#!pip install python-twitter
!pip install TwitterAPI

In [23]:
# Import packages
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from TwitterAPI import TwitterAPI
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation
from bs4 import BeautifulSoup
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords 

In [160]:
data_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/smm2018/data/raw/tweets_harvey_20180218.csv"
train_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/smm2018/data/raw/csv/labelled_edited.csv"

consumer_key = 'FP94BfUftKeygulmcLcVqRNvX'
consumer_secret = 'GfZtNuk6Zu6lTOYGWibXI95MjMyks6SSlBEykyLuYe4NEUgGUu'
access_token_key = '961088185756393472-FopLzpw7n3CrHhbHoWv8BlnR1mZwhGH'
access_token_secret= '62Trp0LVEPvAwoGPH4ov8D4TQe2eEaQETKIfXKCPZX6NN'

In [161]:
api = TwitterAPI(consumer_key, consumer_secret, auth_type='oAuth2')

In [162]:
class ProcessTweets:
    
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['ATUSER','URL','IMG'])
        
    def get_data(query="", source="file", path=data_file, feed="search/tweets",api=api, maxid=0, n=100):
        try:
            if source == "file":
                harvey_df = pd.read_csv(train_file, encoding = 'ISO-8859-1')
                return harvey_df
            else:
                if maxid == 0:
                    return pd.read_json(json.dumps([t for t in api.request(feed, {'q':query,'count':n})]))
                else:
                    return pd.read_json(json.dumps([t for t in api.request(feed, {'q': query, 'count': n, 'max_id': maxid})]))            
        except:
            print("Error while getting data")
            return None
    
    def process_tweets(self, list_of_tweets):
        processed_tweets=[]
        for tweet in list_of_tweets:
            processed_tweets.append((self._process_tweet(tweet)))
        return processed_tweets
    
    def _process_tweet(self,tweet):
        try:
            # Unescape from HTML
            #tweet = html.unescape(tweet)
            tweet = BeautifulSoup(tweet, 'lxml').get_text()
            # 3a. Convert to lower case
            tweet = tweet.lower()
            # 3b. Replace links with the word URL 
            tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet) 
            # 3c. Replace @username with "AT_USER"
            tweet = re.sub('@[^\s]+',' ',tweet)                
            # 3d. Replace #word with word 
            tweet = re.sub(r'#([^\s]+)',r'\1',tweet)
            # 3e. Replace images with the word IMG 
            tweet = re.sub(r'\bpic.twitter.com\s+', ' ', tweet)
            # 3f Keep only words with letters
            tweet = re.sub('[^a-zA-Z]',' ',tweet)
            # 3g. Remove RT
            tweet = re.sub(r'\brt([\b\s])', ' ', tweet)
            # Apply Lemmatization
            lemmatizer = WordNetLemmatizer()           
            tweet = [lemmatizer.lemmatize(word) for word in tweet.split() if word not in self._stopwords and len(word)>1]  
            return (" ".join(tweet)).strip()
        except:
            print("Error with tweet: ", tweet)
            return None

In [163]:
# Process tweets from file
tweet_processor = ProcessTweets()
raw_tweets = tweet_processor.get_data()
#[print(t) for t in raw_tweets['text'][:10]]
cleaned_tweets = tweet_processor.process_tweets(raw_tweets['text'])

In [198]:
# Create train DF 
train_df = pd.DataFrame(cleaned_tweets, columns=['text'])
train_df['target'] = raw_tweets.label
train_df=train_df.drop_duplicates('text')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 417
Data columns (total 2 columns):
text      397 non-null object
target    397 non-null object
dtypes: object(2)
memory usage: 9.3+ KB


In [199]:
# Build a dictionary of feature indices
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(train_df.text)
print(X_train_vec.shape)

tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_vec)
print(X_train_tfidf.shape)

#vectorizer.vocabulary_.get(u'algorithm')

(397, 1253)
(397, 1253)


In [200]:
# Train the classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_df.target)

In [220]:
# Test the classifier
test_tweets = ['find animal', 'My dog is missing please help me find him','we need more volunteers to help', 'Shelters','medical nurse','money','blood','rebuild house']
test_tweets=tweet_processor.process_tweets(test_tweets)

X_test_vec = vectorizer.transform(test_tweets)
X_test_tfidf = tfidf.transform(X_test_vec)

predicted = clf.predict(X_test_tfidf)

predicted
    

array(['Relocation', 'Relocation', 'Volunteering', 'Relocation',
       'Volunteering', 'Money', 'Donation', 'Relocation'],
      dtype='<U14')

In [ ]:
# Evaluation

In [ ]:
# Visualization